# Gurobi Sensitivity Analysis

We consider a general **Linear Programming (LP)** model, which can be used to perform **Sensitivity Analysis** on both **objective coefficients** and **right-hand side (RHS) constraints**.

### General Formulation:
\begin{alignat}{2}
\text{Maximize}   & \quad \sum_{j=1}^{n} c_j x_j & \\
\text{subject to} & \quad \sum_{j=1}^{n} a_{ij} x_j \leq b_i, & \quad \forall i = 1, \ldots, m \\
                  & \quad x_j \geq 0, & \quad \forall j = 1, \ldots, n
\end{alignat}
### Where:
- $ x_j $: Decision variables (e.g., units of products to produce, items to transport).
- $ c_j $: Objective coefficients (profit or cost per unit of \( j \)).
- $ a_{ij} $: Coefficient of variable \( j \) in constraint \( i \) (resource usage).
- $ b_i $: Right-hand side (RHS) of constraint \( i \) (resource availability).
- $ m $: Number of constraints (resources).
- $ n $: Number of decision variables (products, activities, etc.).

---

###  Sensitivity Analysis includes:

- **Objective Coefficient Ranges**: 
  - How much can $ c_j $ increase or decrease without changing the optimal basis/solution.
- **RHS Ranges**:
  - How much can $ b_i $ increase or decrease without changing the shadow prices (dual values).
- **Dual Values (Shadow Prices)**:
  - The value of relaxing a constraint by one unit — interpreted as the **marginal value of resources**.
- **Reduced Costs**:
  - The amount by which an objective coefficient must improve before a non-basic variable enters the solution.

### Libraries 

In [1]:
import gurobipy as gp
from gurobipy import GRB, quicksum, Model
import matplotlib.pyplot as plt
import numpy as np
import time

### Parameters

In [2]:
# Number of products
n_products = 3

# Profits per product
profits = [20, 30, 25]  # Profit for product 1, 2, 3

# Resource consumption per product
# Each row corresponds to a resource: [machine time, labor]
# Each column corresponds to a product
resource_usage = [
    [2, 4, 3],  # Machine time needed for product 1, 2, 3
    [3, 2, 5]   # Labor needed for product 1, 2, 3
]

# Resource availability (right-hand side limits)
resources_available = [100, 90]  # Total machine time, total labor available

# Upper limits on production (optional)
upper_limits = [40, 30, 50]  # Maximum units of products 1, 2, 3

### Create the Gurobi Model

In [3]:
model = Model("ProductionPlanning")

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13


### Variables

In [4]:
x = model.addVars(n_products, lb=0, ub=upper_limits, name="Product")

### Model

In [5]:
# ------------------ Objective ------------------ #
model.setObjective(quicksum(profits[j] * x[j] for j in range(n_products)), GRB.MAXIMIZE)

# Resource constraints
model.addConstrs(
    (quicksum(resource_usage[i][j] * x[j] for j in range(n_products)) <= resources_available[i] 
     for i in range(len(resources_available))),
    name="Resource")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

### Solve the Model

In [6]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x95699edc
Coefficient statistics:
  Matrix range     [2e+00, 5e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [3e+01, 5e+01]
  RHS range        [9e+01, 1e+02]
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+03   6.250000e+00   0.000000e+00      0s
       1    8.5000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  8.500000000e+02


### Display Solution


In [7]:
if model.status == GRB.OPTIMAL:
    print("\n### Optimal Production Plan ###")
    for var in model.getVars():
        print(f"{var.varName}: {var.x:.2f} units")
    print(f"\nTotal Profit: {model.objVal:.2f}")


### Optimal Production Plan ###
Product[0]: 20.00 units
Product[1]: 15.00 units
Product[2]: 0.00 units

Total Profit: 850.00


### Sensitivity Analysis

In [8]:
print("\n### Sensitivity Analysis on Objective Coefficients ###")
print("Variable: Current Profit, Allowable Increase, Allowable Decrease")
for var in model.getVars():
    print(f"{var.varName}: {var.obj}, {var.SAObjUp - var.obj}, {var.obj - var.SAObjLow}")



### Sensitivity Analysis on Objective Coefficients ###
Variable: Current Profit, Allowable Increase, Allowable Decrease
Product[0]: 20.0, 25.0, 3.571428571428573
Product[1]: 30.0, 10.0, 16.666666666666668
Product[2]: 25.0, 6.25, inf


In [9]:
print("\n### Sensitivity Analysis on RHS (Resource Availability) ###")
print("Constraint: Current RHS, Allowable Increase, Allowable Decrease")
for constr in model.getConstrs():
    print(f"{constr.ConstrName}: {constr.RHS}, {constr.SARHSUp - constr.RHS}, {constr.RHS - constr.SARHSLow}")



### Sensitivity Analysis on RHS (Resource Availability) ###
Constraint: Current RHS, Allowable Increase, Allowable Decrease
Resource[0]: 100.0, 40.0, 40.0
Resource[1]: 90.0, 40.0, 40.0


In [10]:
# ------------------ Dual Values ------------------ #
print("\n### Shadow Prices (Dual Values) ###")
for constr in model.getConstrs():
    print(f"{constr.ConstrName}: {constr.pi:.4f}")


### Shadow Prices (Dual Values) ###
Resource[0]: 6.2500
Resource[1]: 2.5000


In [11]:
# ------------------ Reduced Costs ------------------ #
print("\n### Reduced Costs ###")
for var in model.getVars():
    print(f"{var.varName}: {var.RC:.4f}")


### Reduced Costs ###
Product[0]: 0.0000
Product[1]: 0.0000
Product[2]: -6.2500
